<a href="https://www.kaggle.com/code/uniclifeuniclife/dz4cnn?scriptVersionId=166916347" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Домашнее задание к семинару 4.
## Обработка датасета cifar10 при помощи готовой нейронной сети InceptionV3.

In [9]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inc
from tensorflow.keras import layers
from tensorflow.image import resize, grayscale_to_rgb
from tensorflow import cast
from keras import utils
from keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, Dense

In [10]:
batch_size = 32
num_classes = 10
epochs = 5


In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Создание объекта ImageDataGenerator для аугментации данных
datagen = ImageDataGenerator(rescale=1.0/255.0)
x_train = x_train / 255
x_test = x_test / 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [12]:
img_res = 256
# Создание предварительно обученной модели InceptionV3
base_model = inc(weights='imagenet',include_top=False, pooling='avg', input_shape= (img_res,img_res,3))

In [13]:
# Создание генераторов для обучающей и тестовой выборок
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
test_generator = datagen.flow(x_test, y_test, batch_size=batch_size)

In [14]:
model = Sequential()
model.add(UpSampling2D((2,2)))
model.add(UpSampling2D((2,2)))
model.add(UpSampling2D((2,2)))
model.add(base_model)
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [15]:
model.fit(train_generator,
                    epochs=epochs,
                    validation_data=test_generator)

# Оценка производительности модели на тестовых данных
loss, acc = model.evaluate(test_generator, steps=len(x_test) // batch_size)
print('Test loss:', loss)
print('Test accuracy:', acc)

Epoch 1/5
   1/1563 ━━━━━━━━━━━━━━━━━━━━ 16:41:11 38s/step - accuracy: 0.1562 - loss: 2.4129

W0000 00:00:1710384621.081876     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.8121 - loss: 0.5896

W0000 00:00:1710384892.466361     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 326s 184ms/step - accuracy: 0.8121 - loss: 0.5894 - val_accuracy: 0.9245 - val_loss: 0.2226
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 260s 166ms/step - accuracy: 0.9654 - loss: 0.1138 - val_accuracy: 0.8890 - val_loss: 0.3329
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 167ms/step - accuracy: 0.9841 - loss: 0.0529 - val_accuracy: 0.9099 - val_loss: 0.3099
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 168ms/step - accuracy: 0.9920 - loss: 0.0269 - val_accuracy: 0.9480 - val_loss: 0.1739
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 168ms/step - accuracy: 0.9945 - loss: 0.0190 - val_accuracy: 0.9639 - val_loss: 0.1360
312/312 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.9652 - loss: 0.1348
Test loss: 0.13617941737174988
Test accuracy: 0.9640424847602844


# Вывод:
Была использованна сеть InceptionV3 с аугментацией и предварительным увеличением изображения до 256x256 для обработки датасета Cifar10 вместо ResNet50. В итоге InceptionV3 показала гораздо лучшие результаты по сравнению с ResNet50. Точность составила: 0.964, на 5 эпохах.

### Задание 2. Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей можно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.


Для работы с MNIST необходимо сделать:
- загрузить датасет MNIST;
- увеличить размер изображения до 75x75, дублировать каналы до 3:
- def resizer(image,res):
    image = resize(image,(res,res))
    image = image.grayscale_to_rgb(image)
    return image

Для работы с Cifar100 необходимо сделать:
- загрузить датасет Cifar100;
- увеличить размер изображения до 256х256
- увеличить количество нейронов в выходном слое до 100

Для работы с  необходимо сделать:
- загрузить датасет IMAGENET;
- уменьшить размер изображения до 256х256
- увеличить количество нейронов в выходном слое до 1000